In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import pickle
import featuretools as ft

# Load Dataset

In [77]:
df_fkrtl = pd.read_csv("../data/cleaned/fkrtl.csv", parse_dates=["tanggal_datang", "tanggal_pulang"])
df_fkrtl.head()

,no_peserta,no_keluarga,bobot,id_kunjungan_fktp,id_kunjungan_fkrtl,tanggal_datang,tanggal_pulang,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,...,deskripsi_prosthesis,tarif_prosthesis,kode_investigation,deskripsi_investigation,tarif_investigation,kode_drugs,deskripsi_drugs,tarif_drugs,biaya_tagih,biaya_verifikasi
0,50886719,50886719,12.507828,Tidak ada,414390221V002006,2021-02-20,2021-02-20,RIAU,KOTA PEKANBARU,POLRI,...,Tidak ada,0,Tidak ada,Tidak ada,0,Tidak ada,Tidak ada,0,184300,184300
1,70972186,6871133,13.055749,Tidak ada,380700421V000216,2021-04-07,2021-04-08,KALIMANTAN TIMUR,KOTA SAMARINDA,TNI AD,...,Tidak ada,0,Tidak ada,Tidak ada,0,Tidak ada,Tidak ada,0,2578800,2578800
2,31874094,71865092,7.740489,Tidak ada,385450421V006571,2021-04-19,2021-04-19,JAWA TENGAH,KOTA SURAKARTA,Swasta,...,Tidak ada,0,Tidak ada,Tidak ada,0,Tidak ada,Tidak ada,0,193800,193800
3,87945875,87945875,7.734398,Tidak ada,206900221V006045,2021-02-28,2021-03-03,ACEH,ACEH TENGAH,Pemerintah kab/kota,...,Tidak ada,0,Tidak ada,Tidak ada,0,Tidak ada,Tidak ada,0,2705800,2705800
4,384677503,384677503,2.777775,Tidak ada,447120121V000468,2021-01-29,2021-02-01,SUMATERA SELATAN,KOTA PRABUMULIH,Swasta,...,Tidak ada,0,Tidak ada,Tidak ada,0,Tidak ada,Tidak ada,0,2174700,2174700


In [78]:
df_peserta = pd.read_csv("../data/cleaned/peserta.csv", parse_dates=["tanggal_lahir"])
df_peserta.head()

,no_peserta,no_keluarga,tanggal_lahir,hubungan_keluarga,gender,status_perkawinan,kelas_rawat,segmen_peserta,provinsi_tempat_tinggal,kab_kota_tempat_tinggal,kepemilikan_faskes,jenis_faskes,provinsi_faskes,kab_kota_faskes,bobot,tahun_sampel,status_peserta,tahun_meninggal
0,66897393,66897393,1955-12-31,PESERTA,LAKI-LAKI,CERAI,KELAS II,PPU,ACEH,PIDIE,SWASTA,KLINIK PRATAMA,ACEH,PIDIE,11.318326,2021,AKTIF,0
1,36947751,36947751,1957-01-01,PESERTA,LAKI-LAKI,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,KOTA BANDA ACEH,SWASTA,DOKTER UMUM,ACEH,KOTA BANDA ACEH,1.850896,2021,AKTIF,0
2,58468881,58468881,1959-06-15,PESERTA,PEREMPUAN,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,KOTA LANGSA,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,KOTA LANGSA,7.597364,2021,AKTIF,0
3,67318251,67318251,1977-11-11,PESERTA,LAKI-LAKI,KAWIN,KELAS II,PPU,ACEH,KOTA LHOKSEUMAWE,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,KOTA LHOKSEUMAWE,6.003162,2021,AKTIF,0
4,80155656,80155656,1961-12-31,PESERTA,PEREMPUAN,CERAI,KELAS I,PPU,ACEH,BIREUEN,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,BIREUEN,11.383931,2021,AKTIF,0


In [79]:
df_fktp = pd.read_csv("../data/cleaned/fktp-decoded.csv", parse_dates=["tanggal_datang", "tanggal_pulang"])
df_fktp.head()

C:\Users\frend\AppData\Local\Temp\ipykernel_8388\740387771.py:1: DtypeWarning: Columns (3,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fktp = pd.read_csv("../data/cleaned/fktp-decoded.csv", parse_dates=["tanggal_datang", "tanggal_pulang"])


,no_peserta,no_keluarga,bobot,id_kunjungan_fktp,tanggal_datang,tanggal_pulang,provinsi_fktp,kab_kota_fktp,kepemilikan_fktp,jenis_fktp,...,kode_diagnosis_ICD10,kode_diagnosis,nama_diagnosis,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,jenis_fkrtl,tipe_fkrtl,poli_fkrtl,jenis_kunjungan_fktp
0,12380560,12380560,13.364630,1687160334,2021-11-10,2021-11-10,DKI JAKARTA,KOTA JAKARTA BARAT,PEMKAB,PUSKESMAS,...,999,9999,9999,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT
1,288452470,78043385,8.494392,1457720536,2021-02-26,2021-02-26,BENGKULU,REJANG LEBONG,PEMKAB,PUSKESMAS,...,999,9999,9999,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT
2,337885189,337885189,13.472825,1732989007,2021-12-08,2021-12-08,DKI JAKARTA,KOTA JAKARTA UTARA,PEMKAB,PUSKESMAS,...,999,9999,9999,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT
3,2783323,2783323,12.233603,1464029376,2021-03-03,2021-03-03,JAWA BARAT,KOTA DEPOK,PEMKAB,PUSKESMAS,...,999,9999,9999,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT
4,99122276,6688304,6.842236,1646802901,2021-09-28,2021-09-28,NUSA TENGGARA BARAT,KOTA BIMA,PEMKAB,PUSKESMAS,...,999,9999,9999,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT


# Feature Engineering

In [80]:
rawat_inap_values = ['Rawat Inap Kebidanan', 'Rawat Inap Bukan Prosedur']
df_fkrtl = df_fkrtl[df_fkrtl['kelompok_kasus'].isin(rawat_inap_values)].copy()
df_fkrtl = df_fkrtl.sort_values(by=['no_peserta', 'tanggal_datang'])

df_fkrtl['tanggal_kunjungan_berikutnya'] = df_fkrtl.groupby('no_peserta')['tanggal_datang'].shift(-1)

df_fkrtl['jarak_hari_antar_kunjungan'] = (df_fkrtl['tanggal_kunjungan_berikutnya'] - df_fkrtl['tanggal_pulang']).dt.days

df_fkrtl['readmitted_30d'] = np.where(
    (df_fkrtl['jarak_hari_antar_kunjungan'] <= 30) & (df_fkrtl['jarak_hari_antar_kunjungan'] >= 1), 
    1, 
    0
)
df_fkrtl[['no_peserta', 'tanggal_datang', 'tanggal_pulang', 'jarak_hari_antar_kunjungan', 'readmitted_30d']].head(10)

,no_peserta,tanggal_datang,tanggal_pulang,jarak_hari_antar_kunjungan,readmitted_30d
65899,5145,2021-12-16,2021-12-20,NaN,0
27987,6368,2021-07-07,2021-07-11,NaN,0
2943,12638,2021-07-03,2021-07-06,NaN,0
61012,14083,2021-05-15,2021-05-18,NaN,0
69634,17572,2021-01-05,2021-01-08,NaN,0
25351,20972,2021-12-19,2021-12-23,NaN,0
62292,24502,2021-04-29,2021-04-30,30.0,1
62360,24502,2021-05-30,2021-05-31,214.0,0
49777,24502,2021-12-31,2022-01-06,NaN,0
66361,28571,2021-02-10,2021-02-13,46.0,0


In [81]:
df_fkrtl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27827 entries, 65899 to 51791
Data columns (total 58 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   no_peserta                        27827 non-null  int64         
 1   no_keluarga                       27827 non-null  int64         
 2   bobot                             27827 non-null  float64       
 3   id_kunjungan_fktp                 27827 non-null  object        
 4   id_kunjungan_fkrtl                27827 non-null  object        
 5   tanggal_datang                    27827 non-null  datetime64[ns]
 6   tanggal_pulang                    27827 non-null  datetime64[ns]
 7   provinsi_fkrtl                    27827 non-null  object        
 8   kab_kota_fkrtl                    27827 non-null  object        
 9   kepemilikan_fkrtl                 27827 non-null  object        
 10  jenis_fkrtl                       27827 non-nul

In [82]:
df_fkrtl['jarak_hari_antar_kunjungan'] = df_fkrtl['jarak_hari_antar_kunjungan'].fillna(-1)
df_fkrtl = df_fkrtl.drop(columns=['tanggal_kunjungan_berikutnya'])
df_fkrtl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27827 entries, 65899 to 51791
Data columns (total 57 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   no_peserta                        27827 non-null  int64         
 1   no_keluarga                       27827 non-null  int64         
 2   bobot                             27827 non-null  float64       
 3   id_kunjungan_fktp                 27827 non-null  object        
 4   id_kunjungan_fkrtl                27827 non-null  object        
 5   tanggal_datang                    27827 non-null  datetime64[ns]
 6   tanggal_pulang                    27827 non-null  datetime64[ns]
 7   provinsi_fkrtl                    27827 non-null  object        
 8   kab_kota_fkrtl                    27827 non-null  object        
 9   kepemilikan_fkrtl                 27827 non-null  object        
 10  jenis_fkrtl                       27827 non-nul

In [83]:
df_fkrtl['lama_hari_kunjungan'] = df_fkrtl['tanggal_pulang'] - df_fkrtl['tanggal_datang']
df_fkrtl['lama_hari_kunjungan'] = df_fkrtl['lama_hari_kunjungan'].dt.days
df_fkrtl['lama_hari_kunjungan']

65899     4
27987     4
2943      3
61012     3
69634     3
         ..
107797    3
58777     4
9822      3
86366     4
51791     1
Name: lama_hari_kunjungan, Length: 27827, dtype: int64

In [84]:
df_fkrtl['jenis_prosedur']

65899     8749 - Other chest x-ray;9059 - Other microsco...
27987     8628 - Nonexcisional debridement of wound, inf...
2943      9039 - Other microscopic examination of specim...
61012     8744 - Routine chest x-ray, so described;9339 ...
69634                                               Missing
                                ...                        
107797                                              Missing
58777     8703 - Computerized axial tomography of head;8...
9822      8604 - Other incision with drainage of skin an...
86366     3995 - Hemodialysis;8749 - Other chest x-ray;8...
51791     3995 - Hemodialysis;8952 - Electrocardiogram;9...
Name: jenis_prosedur, Length: 27827, dtype: object

In [85]:
jml_kunjungan = df_fkrtl.groupby("no_peserta")['id_kunjungan_fkrtl'].count().reset_index()
jml_kunjungan.columns = ['no_peserta', 'jml_kunjungan_fkrtl']
jml_kunjungan

,no_peserta,jml_kunjungan_fkrtl
0,5145,1
1,6368,1
2,12638,1
3,14083,1
4,17572,1
...,...,...
17989,455126874,1
17990,455143145,4
17991,455196717,1
17992,455254564,1


In [86]:
df_fkrtl

,no_peserta,no_keluarga,bobot,id_kunjungan_fktp,id_kunjungan_fkrtl,tanggal_datang,tanggal_pulang,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,...,deskripsi_investigation,tarif_investigation,kode_drugs,deskripsi_drugs,tarif_drugs,biaya_tagih,biaya_verifikasi,jarak_hari_antar_kunjungan,readmitted_30d,lama_hari_kunjungan
65899,5145,5145,13.006516,Tidak ada,318551221V009825,2021-12-16,2021-12-20,DKI JAKARTA,KOTA JAKARTA BARAT,Swasta,...,Tidak ada,0,Tidak ada,Tidak ada,0,2960400,2960400,-1.0,0,4
27987,6368,6368,14.766182,Tidak ada,316010721V001120,2021-07-07,2021-07-11,JAWA TENGAH,KOTA SEMARANG,Swasta,...,Tidak ada,0,Tidak ada,Tidak ada,0,4389900,4389900,-1.0,0,4
2943,12638,63825415,8.409758,Tidak ada,200430721V001928,2021-07-03,2021-07-06,ACEH,KOTA BANDA ACEH,Pemerintah provinsi,...,Tidak ada,0,Tidak ada,Tidak ada,0,2346400,2346400,-1.0,0,3
61012,14083,8836313,12.324096,Tidak ada,178150521V001878,2021-05-15,2021-05-18,KEPULAUAN BANGKA BELITUNG,BANGKA,Swasta,...,Tidak ada,0,Tidak ada,Tidak ada,0,3085200,3085200,-1.0,0,3
69634,17572,70077067,11.748287,Tidak ada,200440121V000143,2021-01-05,2021-01-08,ACEH,KOTA BANDA ACEH,Swasta,...,Tidak ada,0,Tidak ada,Tidak ada,0,1813300,1813300,-1.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107797,455143145,455143145,8.413463,Tidak ada,101470221V001598,2021-02-13,2021-02-16,JAWA BARAT,KOTA BANJAR,Pemerintah kab/kota,...,Tidak ada,0,Tidak ada,Tidak ada,0,1822000,1822000,-1.0,0,3
58777,455196717,455196717,13.440105,Tidak ada,269840921V001892,2021-09-20,2021-09-24,SUMATERA UTARA,DELI SERDANG,Swasta,...,Tidak ada,0,Tidak ada,Tidak ada,0,4656400,4656400,-1.0,0,4
9822,455254564,365242132,2.503791,Tidak ada,34020321V001980,2021-03-15,2021-03-18,KALIMANTAN TIMUR,PASER,Pemerintah kab/kota,...,Tidak ada,0,Tidak ada,Tidak ada,0,4377100,4377100,-1.0,0,3
86366,455386814,455386814,7.352901,Tidak ada,360310321V003281,2021-03-07,2021-03-11,JAWA TENGAH,KOTA SURAKARTA,Swasta,...,Tidak ada,0,Tidak ada,Tidak ada,0,6312800,6312800,52.0,0,4


In [87]:
df_fkrtl['jml_kunjungan_fkrtl'] = df_fkrtl.groupby("no_peserta").cumcount()
df_fkrtl.head()

,no_peserta,no_keluarga,bobot,id_kunjungan_fktp,id_kunjungan_fkrtl,tanggal_datang,tanggal_pulang,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,...,tarif_investigation,kode_drugs,deskripsi_drugs,tarif_drugs,biaya_tagih,biaya_verifikasi,jarak_hari_antar_kunjungan,readmitted_30d,lama_hari_kunjungan,jml_kunjungan_fkrtl
65899,5145,5145,13.006516,Tidak ada,318551221V009825,2021-12-16,2021-12-20,DKI JAKARTA,KOTA JAKARTA BARAT,Swasta,...,0,Tidak ada,Tidak ada,0,2960400,2960400,-1.0,0,4,0
27987,6368,6368,14.766182,Tidak ada,316010721V001120,2021-07-07,2021-07-11,JAWA TENGAH,KOTA SEMARANG,Swasta,...,0,Tidak ada,Tidak ada,0,4389900,4389900,-1.0,0,4,0
2943,12638,63825415,8.409758,Tidak ada,200430721V001928,2021-07-03,2021-07-06,ACEH,KOTA BANDA ACEH,Pemerintah provinsi,...,0,Tidak ada,Tidak ada,0,2346400,2346400,-1.0,0,3,0
61012,14083,8836313,12.324096,Tidak ada,178150521V001878,2021-05-15,2021-05-18,KEPULAUAN BANGKA BELITUNG,BANGKA,Swasta,...,0,Tidak ada,Tidak ada,0,3085200,3085200,-1.0,0,3,0
69634,17572,70077067,11.748287,Tidak ada,200440121V000143,2021-01-05,2021-01-08,ACEH,KOTA BANDA ACEH,Swasta,...,0,Tidak ada,Tidak ada,0,1813300,1813300,-1.0,0,3,0


In [88]:
df_fkrtl.groupby(["no_peserta"])['kode_diagnosis_ICD10'].nunique()

no_peserta
5145         1
6368         1
12638        1
14083        1
17572        1
            ..
455126874    1
455143145    1
455196717    1
455254564    1
455386814    2
Name: kode_diagnosis_ICD10, Length: 17994, dtype: int64

In [89]:
jml_diagnosis = df_fkrtl.groupby("no_peserta")['kode_diagnosis_ICD10'].nunique().reset_index()
jml_diagnosis.columns = ['no_peserta', 'jml_diagnosis']
jml_diagnosis

,no_peserta,jml_diagnosis
0,5145,1
1,6368,1
2,12638,1
3,14083,1
4,17572,1
...,...,...
17989,455126874,1
17990,455143145,1
17991,455196717,1
17992,455254564,1


In [90]:
df_peserta = df_peserta.merge(
    jml_diagnosis,
    on="no_peserta",
    how="left"
)
df_peserta.head()

,no_peserta,no_keluarga,tanggal_lahir,hubungan_keluarga,gender,status_perkawinan,kelas_rawat,segmen_peserta,provinsi_tempat_tinggal,kab_kota_tempat_tinggal,kepemilikan_faskes,jenis_faskes,provinsi_faskes,kab_kota_faskes,bobot,tahun_sampel,status_peserta,tahun_meninggal,jml_diagnosis
0,66897393,66897393,1955-12-31,PESERTA,LAKI-LAKI,CERAI,KELAS II,PPU,ACEH,PIDIE,SWASTA,KLINIK PRATAMA,ACEH,PIDIE,11.318326,2021,AKTIF,0,NaN
1,36947751,36947751,1957-01-01,PESERTA,LAKI-LAKI,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,KOTA BANDA ACEH,SWASTA,DOKTER UMUM,ACEH,KOTA BANDA ACEH,1.850896,2021,AKTIF,0,NaN
2,58468881,58468881,1959-06-15,PESERTA,PEREMPUAN,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,KOTA LANGSA,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,KOTA LANGSA,7.597364,2021,AKTIF,0,1.0
3,67318251,67318251,1977-11-11,PESERTA,LAKI-LAKI,KAWIN,KELAS II,PPU,ACEH,KOTA LHOKSEUMAWE,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,KOTA LHOKSEUMAWE,6.003162,2021,AKTIF,0,NaN
4,80155656,80155656,1961-12-31,PESERTA,PEREMPUAN,CERAI,KELAS I,PPU,ACEH,BIREUEN,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,BIREUEN,11.383931,2021,AKTIF,0,NaN


In [91]:
avg_hari_kunjungan = df_fkrtl.groupby("no_peserta")['lama_hari_kunjungan'].mean().reset_index()
avg_hari_kunjungan.columns = ['no_peserta', 'avg_hari_kunjungan_fkrtl']
avg_hari_kunjungan

,no_peserta,avg_hari_kunjungan_fkrtl
0,5145,4.00
1,6368,4.00
2,12638,3.00
3,14083,3.00
4,17572,3.00
...,...,...
17989,455126874,4.00
17990,455143145,2.25
17991,455196717,4.00
17992,455254564,3.00


In [92]:
df_peserta = df_peserta.merge(
    avg_hari_kunjungan,
    on="no_peserta",
    how="left"
)
df_peserta.head()

,no_peserta,no_keluarga,tanggal_lahir,hubungan_keluarga,gender,status_perkawinan,kelas_rawat,segmen_peserta,provinsi_tempat_tinggal,kab_kota_tempat_tinggal,kepemilikan_faskes,jenis_faskes,provinsi_faskes,kab_kota_faskes,bobot,tahun_sampel,status_peserta,tahun_meninggal,jml_diagnosis,avg_hari_kunjungan_fkrtl
0,66897393,66897393,1955-12-31,PESERTA,LAKI-LAKI,CERAI,KELAS II,PPU,ACEH,PIDIE,SWASTA,KLINIK PRATAMA,ACEH,PIDIE,11.318326,2021,AKTIF,0,NaN,NaN
1,36947751,36947751,1957-01-01,PESERTA,LAKI-LAKI,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,KOTA BANDA ACEH,SWASTA,DOKTER UMUM,ACEH,KOTA BANDA ACEH,1.850896,2021,AKTIF,0,NaN,NaN
2,58468881,58468881,1959-06-15,PESERTA,PEREMPUAN,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,KOTA LANGSA,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,KOTA LANGSA,7.597364,2021,AKTIF,0,1.0,4.0
3,67318251,67318251,1977-11-11,PESERTA,LAKI-LAKI,KAWIN,KELAS II,PPU,ACEH,KOTA LHOKSEUMAWE,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,KOTA LHOKSEUMAWE,6.003162,2021,AKTIF,0,NaN,NaN
4,80155656,80155656,1961-12-31,PESERTA,PEREMPUAN,CERAI,KELAS I,PPU,ACEH,BIREUEN,PEMERINTAH KABUPATEN/KOTA,PUSKESMAS,ACEH,BIREUEN,11.383931,2021,AKTIF,0,NaN,NaN


In [93]:
df_fktp = df_fktp.sort_values(by=['no_peserta', 'tanggal_datang'])

df_fktp['tanggal_kunjungan_berikutnya'] = df_fktp.groupby('no_peserta')['tanggal_datang'].shift(-1)

df_fktp['jarak_hari_antar_kunjungan'] = (df_fktp['tanggal_kunjungan_berikutnya'] - df_fktp['tanggal_pulang']).dt.days
df_fktp.head()

,no_peserta,no_keluarga,bobot,id_kunjungan_fktp,tanggal_datang,tanggal_pulang,provinsi_fktp,kab_kota_fktp,kepemilikan_fktp,jenis_fktp,...,nama_diagnosis,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,jenis_fkrtl,tipe_fkrtl,poli_fkrtl,jenis_kunjungan_fktp,tanggal_kunjungan_berikutnya,jarak_hari_antar_kunjungan
231085,968,968,13.006498,1427216889,2021-01-22,2021-01-22,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,9999,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT,2021-02-16,25.0
166234,968,968,13.006498,1447536743,2021-02-16,2021-02-16,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,9999,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT,2021-03-24,36.0
303049,968,968,13.006498,386310321Y001174,2021-03-24,2021-03-24,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,Insulin-dependent diabetes mellitus with unspe...,JAWA TIMUR,KOTA SURABAYA,PEMERINTAH KABUPATEN/KOTA,RUMAH SAKIT,RS Kelas B,PENYAKIT DALAM,KUNJUNGAN SAKIT,2021-06-23,91.0
281731,968,968,13.006498,386310621Y001373,2021-06-23,2021-06-23,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,Insulin-dependent diabetes mellitus with unspe...,JAWA TIMUR,KOTA SURABAYA,PEMERINTAH KABUPATEN/KOTA,RUMAH SAKIT,RS Kelas B,PENYAKIT DALAM,KUNJUNGAN SAKIT,2021-09-24,93.0
268159,968,968,13.006498,386310921Y001008,2021-09-24,2021-09-24,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,Insulin-dependent diabetes mellitus with unspe...,JAWA TIMUR,KOTA SURABAYA,PEMERINTAH KABUPATEN/KOTA,RUMAH SAKIT,RS Kelas B,PENYAKIT DALAM,KUNJUNGAN SAKIT,2021-12-27,94.0


In [94]:
df_fktp['jml_kunjungan_fktp'] = df_fktp.groupby("no_peserta").cumcount()
df_fktp.head()

,no_peserta,no_keluarga,bobot,id_kunjungan_fktp,tanggal_datang,tanggal_pulang,provinsi_fktp,kab_kota_fktp,kepemilikan_fktp,jenis_fktp,...,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,jenis_fkrtl,tipe_fkrtl,poli_fkrtl,jenis_kunjungan_fktp,tanggal_kunjungan_berikutnya,jarak_hari_antar_kunjungan,jml_kunjungan_fktp
231085,968,968,13.006498,1427216889,2021-01-22,2021-01-22,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT,2021-02-16,25.0,0
166234,968,968,13.006498,1447536743,2021-02-16,2021-02-16,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,TIDAK DIRUJUK,KUNJUNGAN SEHAT,2021-03-24,36.0,1
303049,968,968,13.006498,386310321Y001174,2021-03-24,2021-03-24,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,JAWA TIMUR,KOTA SURABAYA,PEMERINTAH KABUPATEN/KOTA,RUMAH SAKIT,RS Kelas B,PENYAKIT DALAM,KUNJUNGAN SAKIT,2021-06-23,91.0,2
281731,968,968,13.006498,386310621Y001373,2021-06-23,2021-06-23,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,JAWA TIMUR,KOTA SURABAYA,PEMERINTAH KABUPATEN/KOTA,RUMAH SAKIT,RS Kelas B,PENYAKIT DALAM,KUNJUNGAN SAKIT,2021-09-24,93.0,3
268159,968,968,13.006498,386310921Y001008,2021-09-24,2021-09-24,JAWA TIMUR,KOTA SURABAYA,PEMKAB,PUSKESMAS,...,JAWA TIMUR,KOTA SURABAYA,PEMERINTAH KABUPATEN/KOTA,RUMAH SAKIT,RS Kelas B,PENYAKIT DALAM,KUNJUNGAN SAKIT,2021-12-27,94.0,4


In [95]:
df_peserta = df_peserta.merge(
    jml_kunjungan,
    on="no_peserta",
    how="left"
)
df_peserta.head()

,no_peserta,no_keluarga,tanggal_lahir,hubungan_keluarga,gender,status_perkawinan,kelas_rawat,segmen_peserta,provinsi_tempat_tinggal,kab_kota_tempat_tinggal,...,jenis_faskes,provinsi_faskes,kab_kota_faskes,bobot,tahun_sampel,status_peserta,tahun_meninggal,jml_diagnosis,avg_hari_kunjungan_fkrtl,jml_kunjungan_fkrtl
0,66897393,66897393,1955-12-31,PESERTA,LAKI-LAKI,CERAI,KELAS II,PPU,ACEH,PIDIE,...,KLINIK PRATAMA,ACEH,PIDIE,11.318326,2021,AKTIF,0,NaN,NaN,NaN
1,36947751,36947751,1957-01-01,PESERTA,LAKI-LAKI,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,KOTA BANDA ACEH,...,DOKTER UMUM,ACEH,KOTA BANDA ACEH,1.850896,2021,AKTIF,0,NaN,NaN,NaN
2,58468881,58468881,1959-06-15,PESERTA,PEREMPUAN,KAWIN,KELAS I,BUKAN PEKERJA,ACEH,KOTA LANGSA,...,PUSKESMAS,ACEH,KOTA LANGSA,7.597364,2021,AKTIF,0,1.0,4.0,1.0
3,67318251,67318251,1977-11-11,PESERTA,LAKI-LAKI,KAWIN,KELAS II,PPU,ACEH,KOTA LHOKSEUMAWE,...,PUSKESMAS,ACEH,KOTA LHOKSEUMAWE,6.003162,2021,AKTIF,0,NaN,NaN,NaN
4,80155656,80155656,1961-12-31,PESERTA,PEREMPUAN,CERAI,KELAS I,PPU,ACEH,BIREUEN,...,PUSKESMAS,ACEH,BIREUEN,11.383931,2021,AKTIF,0,NaN,NaN,NaN


In [96]:
df_merged = df_fkrtl.merge(
    df_peserta,
    on=["no_peserta", "bobot", "segmen_peserta"],
    how="left",
    suffixes=("", "_drop")
)

df_merged = df_merged.drop(columns=[col for col in df_merged.columns if col.endswith("_drop")])

In [97]:
df_merged = df_merged.merge(
    df_fktp,
    on="id_kunjungan_fktp",
    how="left",
    suffixes=("", "_drop")
)
df_merged = df_merged.drop(columns=[col for col in df_merged.columns if col.endswith("_drop")])

In [98]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27827 entries, 0 to 27826
Data columns (total 84 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   no_peserta                        27827 non-null  int64         
 1   no_keluarga                       27827 non-null  int64         
 2   bobot                             27827 non-null  float64       
 3   id_kunjungan_fktp                 27827 non-null  object        
 4   id_kunjungan_fkrtl                27827 non-null  object        
 5   tanggal_datang                    27827 non-null  datetime64[ns]
 6   tanggal_pulang                    27827 non-null  datetime64[ns]
 7   provinsi_fkrtl                    27827 non-null  object        
 8   kab_kota_fkrtl                    27827 non-null  object        
 9   kepemilikan_fkrtl                 27827 non-null  object        
 10  jenis_fkrtl                       27827 non-nu

In [99]:
df_merged.head()

,no_peserta,no_keluarga,bobot,id_kunjungan_fktp,id_kunjungan_fkrtl,tanggal_datang,tanggal_pulang,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,...,avg_hari_kunjungan_fkrtl,provinsi_fktp,kab_kota_fktp,kepemilikan_fktp,jenis_fktp,tipe_fktp,poli_fktp,jenis_kunjungan_fktp,tanggal_kunjungan_berikutnya,jml_kunjungan_fktp
0,5145,5145,13.006516,Tidak ada,318551221V009825,2021-12-16,2021-12-20,DKI JAKARTA,KOTA JAKARTA BARAT,Swasta,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,6368,6368,14.766182,Tidak ada,316010721V001120,2021-07-07,2021-07-11,JAWA TENGAH,KOTA SEMARANG,Swasta,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,12638,63825415,8.409758,Tidak ada,200430721V001928,2021-07-03,2021-07-06,ACEH,KOTA BANDA ACEH,Pemerintah provinsi,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,14083,8836313,12.324096,Tidak ada,178150521V001878,2021-05-15,2021-05-18,KEPULAUAN BANGKA BELITUNG,BANGKA,Swasta,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,17572,70077067,11.748287,Tidak ada,200440121V000143,2021-01-05,2021-01-08,ACEH,KOTA BANDA ACEH,Swasta,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [100]:
df_fkrtl_no_target = df_fkrtl.drop(columns=['readmitted_30d'])
df_fkrtl_2 = df_fkrtl.drop(columns=['jarak_hari_antar_kunjungan'])

# Automated Feature Engineering

## Generating new Engineered Columns

In [101]:
es = ft.EntitySet(id="bpjs_data")

es = es.add_dataframe(
    dataframe_name="peserta",
    dataframe=df_peserta,
    index="no_peserta"
)

es = es.add_dataframe(
    dataframe_name="fkrtl", 
    dataframe=df_fkrtl_2,
    index="id_kunjungan_fkrtl",
    time_index="tanggal_datang"
)

es = es.add_dataframe(
    dataframe_name="fktp",
    dataframe=df_fktp,
    index="id_kunjungan_fktp", 
    time_index="tanggal_datang"
)

es = es.add_relationship("peserta", "no_peserta", "fkrtl", "no_peserta")
es = es.add_relationship("peserta", "no_peserta", "fktp", "no_peserta")

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="fkrtl",
    agg_primitives=["sum", "mean", "max", "min", "std", "count"],
    trans_primitives=["day", "month", "year", "weekday"],
    max_depth=2
)

print(f"Generated {len(feature_defs)} features")

c:\Users\frend\Projects\Classification\hospital_readmission_prediction\venv\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\frend\Projects\Classification\hospital_readmission_prediction\venv\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\frend\Projects\Classification\hospital_readmission_prediction\venv\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
c:\Users\frend\Proj

Generated 177 features


In [102]:
for column in feature_matrix.columns:
    print(column)

no_peserta
no_keluarga
bobot
id_kunjungan_fktp
provinsi_fkrtl
kab_kota_fkrtl
kepemilikan_fkrtl
jenis_fkrtl
tipe_fkrtl
tingkat_pelayanan
poli_fkrtl
segmen_peserta
kelas_iuran_premi
status_pulang_peserta
kode_nama_diagnosis_ICD10
kode_diagnosis_ICD10
kode_diagnosis
nama_diagnosis
kode_nama_diagnosis_primer_ICD10
kode_diagnosis_primer_ICD10
kode_diagnosis_primer
nama_diagnosis_primer
kode_INACBGs
deskripsi_INACBGs
kode_casemix
kelompok_kasus
spesifikasi_kelompok_kasus
tingkat_keparahan_kelompok_kasus
provinsi_perujuk
kab_kota_perujuk
kepemilikan_perujuk
jenis_perujuk
tipe_perujuk
tarif_regional_INACBGs
group_tarif_INACBGs
kode_sub_acute
tarif_sub_acute
kode_procedures
deskripsi_procedures
tarif_procedures
kode_prosthesis
deskripsi_prosthesis
tarif_prosthesis
kode_investigation
deskripsi_investigation
tarif_investigation
kode_drugs
deskripsi_drugs
tarif_drugs
biaya_tagih
biaya_verifikasi
readmitted_30d
lama_hari_kunjungan
jml_kunjungan_fkrtl
DAY(tanggal_datang)
DAY(tanggal_pulang)
MONTH(ta

In [103]:
feature_matrix.to_csv("../data/interim/feature_matrix.csv", index=False)

## Removing Highly Correlated Columns

In [104]:
correlation_matrix = feature_matrix.corr(numeric_only=True).abs()
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)

to_drop = [column for column in upper_triangle.columns 
           if any(upper_triangle[column] > 0.95)]

print(f"Dropping {len(to_drop)} highly correlated features")
feature_matrix_reduced = feature_matrix.drop(columns=to_drop)

Dropping 46 highly correlated features


In [105]:
for column in feature_matrix_reduced.columns:
    print(column)

no_peserta
no_keluarga
bobot
id_kunjungan_fktp
provinsi_fkrtl
kab_kota_fkrtl
kepemilikan_fkrtl
jenis_fkrtl
tipe_fkrtl
tingkat_pelayanan
poli_fkrtl
segmen_peserta
kelas_iuran_premi
status_pulang_peserta
kode_nama_diagnosis_ICD10
kode_diagnosis_ICD10
kode_diagnosis
nama_diagnosis
kode_nama_diagnosis_primer_ICD10
kode_diagnosis_primer_ICD10
kode_diagnosis_primer
nama_diagnosis_primer
kode_INACBGs
deskripsi_INACBGs
kode_casemix
kelompok_kasus
spesifikasi_kelompok_kasus
tingkat_keparahan_kelompok_kasus
provinsi_perujuk
kab_kota_perujuk
kepemilikan_perujuk
jenis_perujuk
tipe_perujuk
tarif_regional_INACBGs
group_tarif_INACBGs
kode_sub_acute
tarif_sub_acute
kode_procedures
deskripsi_procedures
tarif_procedures
kode_prosthesis
deskripsi_prosthesis
tarif_prosthesis
kode_investigation
deskripsi_investigation
tarif_investigation
kode_drugs
deskripsi_drugs
tarif_drugs
readmitted_30d
lama_hari_kunjungan
jml_kunjungan_fkrtl
DAY(tanggal_datang)
DAY(tanggal_pulang)
MONTH(tanggal_datang)
MONTH(tanggal_p

In [106]:
feature_matrix_reduced.head()

,no_peserta,no_keluarga,bobot,id_kunjungan_fktp,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,jenis_fkrtl,tipe_fkrtl,tingkat_pelayanan,...,peserta.STD(fktp.jarak_hari_antar_kunjungan),peserta.STD(fktp.no_keluarga),peserta.SUM(fktp.bobot),peserta.SUM(fktp.jarak_hari_antar_kunjungan),peserta.SUM(fktp.jml_kunjungan_fktp),peserta.SUM(fktp.no_keluarga),peserta.DAY(tanggal_lahir),peserta.MONTH(tanggal_lahir),peserta.WEEKDAY(tanggal_lahir),peserta.YEAR(tanggal_lahir)
id_kunjungan_fkrtl,,,,,,,,,,,,,,,,,,,,,
106840121V000074,6589541,6589541,1.309261,Tidak ada,LAMPUNG,TULANGBAWANG,Pemerintah kab/kota,Rumah sakit,RS Kelas B,RITL,...,35.008842,0.0,10.474086,347.0,28.0,52716328.0,4,6,4,1965
107070121V000066,75958436,67665370,12.408093,Tidak ada,BALI,KOTA DENPASAR,Pemerintah kab/kota,Rumah sakit,RS Kelas B,RITL,...,29.739798,0.0,136.489023,327.0,55.0,744319070.0,31,12,0,1956
125360121V000104,3839008,3839008,12.996570,Tidak ada,JAWA TIMUR,SIDOARJO,Pemerintah kab/kota,Rumah sakit,RS Kelas B,RITL,...,12.987881,0.0,402.893670,338.0,465.0,119009248.0,12,6,1,1945
151020121V000038,26836996,26836996,8.875113,Tidak ada,SULAWESI SELATAN,LUWU,Swasta,Rumah sakit,RS Kelas C,RITL,...,23.362125,0.0,97.626248,307.0,55.0,295206956.0,2,9,1,1975
155190121V000021,29400147,54114726,3.093151,Tidak ada,ACEH,KOTA SUBULUSSALAM,Pemerintah kab/kota,Rumah sakit,RS Kelas C,RITL,...,35.711343,0.0,18.558904,308.0,15.0,324688356.0,1,12,5,1973


In [107]:
feature_matrix_reduced.isnull().sum()

no_peserta                        0
no_keluarga                       0
bobot                             0
id_kunjungan_fktp                 0
provinsi_fkrtl                    0
                                 ..
peserta.SUM(fktp.no_keluarga)     0
peserta.DAY(tanggal_lahir)        0
peserta.MONTH(tanggal_lahir)      0
peserta.WEEKDAY(tanggal_lahir)    0
peserta.YEAR(tanggal_lahir)       0
Length: 131, dtype: int64

# Feature Importance

In [108]:
features = feature_matrix_reduced.drop(columns=['no_peserta', 'no_keluarga'])
features.head()

,bobot,id_kunjungan_fktp,provinsi_fkrtl,kab_kota_fkrtl,kepemilikan_fkrtl,jenis_fkrtl,tipe_fkrtl,tingkat_pelayanan,poli_fkrtl,segmen_peserta,...,peserta.STD(fktp.jarak_hari_antar_kunjungan),peserta.STD(fktp.no_keluarga),peserta.SUM(fktp.bobot),peserta.SUM(fktp.jarak_hari_antar_kunjungan),peserta.SUM(fktp.jml_kunjungan_fktp),peserta.SUM(fktp.no_keluarga),peserta.DAY(tanggal_lahir),peserta.MONTH(tanggal_lahir),peserta.WEEKDAY(tanggal_lahir),peserta.YEAR(tanggal_lahir)
id_kunjungan_fkrtl,,,,,,,,,,,,,,,,,,,,,
106840121V000074,1.309261,Tidak ada,LAMPUNG,TULANGBAWANG,Pemerintah kab/kota,Rumah sakit,RS Kelas B,RITL,Missing,PBI APBD,...,35.008842,0.0,10.474086,347.0,28.0,52716328.0,4,6,4,1965
107070121V000066,12.408093,Tidak ada,BALI,KOTA DENPASAR,Pemerintah kab/kota,Rumah sakit,RS Kelas B,RITL,Missing,PBPU,...,29.739798,0.0,136.489023,327.0,55.0,744319070.0,31,12,0,1956
125360121V000104,12.996570,Tidak ada,JAWA TIMUR,SIDOARJO,Pemerintah kab/kota,Rumah sakit,RS Kelas B,RITL,Missing,Bukan pekerja,...,12.987881,0.0,402.893670,338.0,465.0,119009248.0,12,6,1,1945
151020121V000038,8.875113,Tidak ada,SULAWESI SELATAN,LUWU,Swasta,Rumah sakit,RS Kelas C,RITL,Missing,PBI APBN,...,23.362125,0.0,97.626248,307.0,55.0,295206956.0,2,9,1,1975
155190121V000021,3.093151,Tidak ada,ACEH,KOTA SUBULUSSALAM,Pemerintah kab/kota,Rumah sakit,RS Kelas C,RITL,Missing,PBI APBN,...,35.711343,0.0,18.558904,308.0,15.0,324688356.0,1,12,5,1973


In [109]:
df_merged_2 = df_merged.drop(columns=['no_peserta', 'no_keluarga', 'id_kunjungan_fktp', 'id_kunjungan_fkrtl', 'tanggal_lahir', 'tanggal_datang', 'tanggal_pulang', 'tanggal_kunjungan_berikutnya', 'jarak_hari_antar_kunjungan'])

encoder = LabelEncoder()
cat_cols = df_merged_2.select_dtypes("object").columns
for col in cat_cols:
    df_merged_2[col] = encoder.fit_transform(df_merged_2[col])

X = df_merged_2.drop(columns=['readmitted_30d'])
y = df_merged_2[['readmitted_30d']]

In [120]:
cat_cols = features.select_dtypes("category").columns
num_cols = features.select_dtypes("number").columns
for col in cat_cols:
    features[col] = features[col].cat.codes

X = features.drop(columns=[
        'readmitted_30d', 
        'peserta.MEAN(fkrtl.readmitted_30d)',
        'peserta.MAX(fkrtl.readmitted_30d)',
        'peserta.SUM(fkrtl.readmitted_30d)',
        'peserta.jml_kunjungan_fkrtl'
    ])
y = features[['readmitted_30d']]

In [121]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = XGBClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

top_features = feature_importance.head(50)['feature'].tolist()

In [122]:
model.score(X_test, y_test)

0.888609414301114

In [123]:
feature_importance.head(20)

,feature,importance
96,peserta.STD(fkrtl.lama_hari_kunjungan),0.164362
103,peserta.SUM(fkrtl.jml_kunjungan_fkrtl),0.149191
48,jml_kunjungan_fkrtl,0.053215
11,status_pulang_peserta,0.025396
52,MONTH(tanggal_pulang),0.023726
56,YEAR(tanggal_pulang),0.023651
104,peserta.SUM(fkrtl.lama_hari_kunjungan),0.021895
69,peserta.status_peserta,0.021510
117,peserta.SUM(fktp.jarak_hari_antar_kunjungan),0.017365
51,MONTH(tanggal_datang),0.015404


In [124]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      4716
           1       0.65      0.60      0.62       850

    accuracy                           0.89      5566
   macro avg       0.79      0.77      0.78      5566
weighted avg       0.89      0.89      0.89      5566



In [125]:
features.to_csv("../data/interim/all_features.csv")

# Feature Store Peserta

In [126]:
list(feature_importance.head(40).feature)

['peserta.STD(fkrtl.lama_hari_kunjungan)',
 'peserta.SUM(fkrtl.jml_kunjungan_fkrtl)',
 'jml_kunjungan_fkrtl',
 'status_pulang_peserta',
 'MONTH(tanggal_pulang)',
 'YEAR(tanggal_pulang)',
 'peserta.SUM(fkrtl.lama_hari_kunjungan)',
 'peserta.status_peserta',
 'peserta.SUM(fktp.jarak_hari_antar_kunjungan)',
 'MONTH(tanggal_datang)',
 'kode_nama_diagnosis_primer_ICD10',
 'kepemilikan_perujuk',
 'peserta.provinsi_faskes',
 'kode_casemix',
 'jenis_perujuk',
 'tipe_perujuk',
 'peserta.MEAN(fktp.jarak_hari_antar_kunjungan)',
 'peserta.segmen_peserta',
 'kepemilikan_fkrtl',
 'peserta.STD(fkrtl.spesifikasi_kelompok_kasus)',
 'kode_nama_diagnosis_ICD10',
 'peserta.MAX(fkrtl.tarif_drugs)',
 'peserta.kab_kota_tempat_tinggal',
 'kode_INACBGs',
 'tipe_fkrtl',
 'peserta.provinsi_tempat_tinggal',
 'peserta.SUM(fktp.no_keluarga)',
 'peserta.MEAN(fkrtl.spesifikasi_kelompok_kasus)',
 'peserta.SUM(fkrtl.biaya_tagih)',
 'kab_kota_fkrtl',
 'peserta.MAX(fktp.jarak_hari_antar_kunjungan)',
 'peserta.SUM(fkrtl.n

In [129]:
feature_store_cols = [
    'peserta.STD(fkrtl.lama_hari_kunjungan)',
    'peserta.SUM(fkrtl.jml_kunjungan_fkrtl)',
    'peserta.SUM(fkrtl.lama_hari_kunjungan)',
    'peserta.status_peserta',
    'peserta.SUM(fktp.jarak_hari_antar_kunjungan)',
    'peserta.provinsi_faskes',
    'peserta.MEAN(fktp.jarak_hari_antar_kunjungan)',
    'peserta.segmen_peserta',
    'peserta.STD(fkrtl.spesifikasi_kelompok_kasus)',
    'peserta.MAX(fkrtl.tarif_drugs)',
    'peserta.kab_kota_tempat_tinggal',
    'peserta.provinsi_tempat_tinggal',
    'peserta.SUM(fktp.no_keluarga)',
    'peserta.MEAN(fkrtl.spesifikasi_kelompok_kasus)',
    'peserta.SUM(fkrtl.biaya_tagih)',
    'peserta.MAX(fktp.jarak_hari_antar_kunjungan)',
    'peserta.SUM(fkrtl.no_keluarga)',
    'peserta.MIN(fkrtl.spesifikasi_kelompok_kasus)',
    'peserta.SUM(fkrtl.bobot)',
    'peserta.STD(fkrtl.biaya_tagih)',
    'peserta.SUM(fktp.bobot)',
    'peserta.MIN(fkrtl.biaya_tagih)',
    'peserta.gender',
    'peserta.COUNT(fktp)'
]

In [130]:
feature_matrix_peserta = features[feature_store_cols]
feature_matrix_peserta['no_peserta'] = feature_matrix['no_peserta']

In [131]:
feature_matrix_peserta

,peserta.STD(fkrtl.lama_hari_kunjungan),peserta.SUM(fkrtl.jml_kunjungan_fkrtl),peserta.SUM(fkrtl.lama_hari_kunjungan),peserta.status_peserta,peserta.SUM(fktp.jarak_hari_antar_kunjungan),peserta.provinsi_faskes,peserta.MEAN(fktp.jarak_hari_antar_kunjungan),peserta.segmen_peserta,peserta.STD(fkrtl.spesifikasi_kelompok_kasus),peserta.MAX(fkrtl.tarif_drugs),...,peserta.MAX(fktp.jarak_hari_antar_kunjungan),peserta.SUM(fkrtl.no_keluarga),peserta.MIN(fkrtl.spesifikasi_kelompok_kasus),peserta.SUM(fkrtl.bobot),peserta.STD(fkrtl.biaya_tagih),peserta.SUM(fktp.bobot),peserta.MIN(fkrtl.biaya_tagih),peserta.gender,peserta.COUNT(fktp),no_peserta
id_kunjungan_fkrtl,,,,,,,,,,,,,,,,,,,,,
106840121V000074,1.000000,3.0,12.0,1,347.0,18,49.571429,2,3.464102,0.0,...,112.0,19768623.0,10.0,3.927782,1.148927e+04,10.474086,3411400.0,1,8,6589541
107070121V000066,6.082763,3.0,15.0,1,327.0,1,32.700000,4,6.506407,0.0,...,75.0,202996110.0,10.0,37.224279,2.465712e+06,136.489023,3603800.0,1,11,75958436
125360121V000104,0.707107,1.0,7.0,1,338.0,10,11.266667,1,2.121320,0.0,...,49.0,7678016.0,17.0,25.993140,9.607460e+05,402.893670,4296500.0,0,31,3839008
151020121V000038,1.414214,1.0,6.0,1,307.0,27,30.700000,3,0.707107,0.0,...,71.0,53673992.0,17.0,17.750227,1.545806e+06,97.626248,1295200.0,1,11,26836996
155190121V000021,1.258306,6.0,13.0,1,308.0,0,61.600000,3,0.500000,0.0,...,88.0,216458904.0,15.0,12.372602,3.375500e+05,18.558904,3372600.0,1,6,29400147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64851221V000333,2.121320,1.0,11.0,1,221.0,5,221.000000,2,4.242641,0.0,...,221.0,853963036.0,10.0,20.127734,2.899845e+05,20.127734,5077500.0,0,2,426981518
65631221V020785,NaN,0.0,5.0,1,277.0,5,46.166667,5,NaN,0.0,...,86.0,21121422.0,10.0,13.297376,NaN,93.081632,8329600.0,1,7,21121422
731741221V000485,NaN,0.0,2.0,1,336.0,18,12.923077,4,NaN,0.0,...,44.0,96405979.0,17.0,10.350811,NaN,279.471897,4551200.0,1,27,72859783


In [132]:
feature_store_peserta = feature_matrix_peserta.groupby("no_peserta")[feature_store_cols].mean()

feature_store_peserta.head()

,peserta.STD(fkrtl.lama_hari_kunjungan),peserta.SUM(fkrtl.jml_kunjungan_fkrtl),peserta.SUM(fkrtl.lama_hari_kunjungan),peserta.status_peserta,peserta.SUM(fktp.jarak_hari_antar_kunjungan),peserta.provinsi_faskes,peserta.MEAN(fktp.jarak_hari_antar_kunjungan),peserta.segmen_peserta,peserta.STD(fkrtl.spesifikasi_kelompok_kasus),peserta.MAX(fkrtl.tarif_drugs),...,peserta.SUM(fkrtl.biaya_tagih),peserta.MAX(fktp.jarak_hari_antar_kunjungan),peserta.SUM(fkrtl.no_keluarga),peserta.MIN(fkrtl.spesifikasi_kelompok_kasus),peserta.SUM(fkrtl.bobot),peserta.STD(fkrtl.biaya_tagih),peserta.SUM(fktp.bobot),peserta.MIN(fkrtl.biaya_tagih),peserta.gender,peserta.COUNT(fktp)
no_peserta,,,,,,,,,,,,,,,,,,,,,
5145,NaN,0.0,4.0,1.0,322.0,5.0,15.333333,5.0,NaN,0.0,...,2960400.0,62.0,5145.0,18.0,13.006516,NaN,286.143341,2960400.0,0.0,22.0
6368,NaN,0.0,4.0,1.0,260.0,9.0,65.000000,3.0,NaN,0.0,...,4389900.0,142.0,6368.0,15.0,14.766182,NaN,73.830910,4389900.0,0.0,5.0
12638,NaN,0.0,3.0,1.0,294.0,0.0,58.800000,2.0,NaN,0.0,...,2346400.0,101.0,63825415.0,14.0,8.409758,NaN,50.458548,2346400.0,1.0,6.0
14083,NaN,0.0,3.0,1.0,321.0,16.0,16.050000,4.0,NaN,0.0,...,3085200.0,44.0,8836313.0,26.0,12.324096,NaN,258.806016,3085200.0,1.0,21.0
17572,NaN,0.0,3.0,1.0,309.0,0.0,61.800000,5.0,NaN,0.0,...,1813300.0,100.0,70077067.0,18.0,11.748287,NaN,70.489722,1813300.0,1.0,6.0


In [133]:
feature_store_peserta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17994 entries, 5145 to 455386814
Data columns (total 24 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   peserta.STD(fkrtl.lama_hari_kunjungan)          5416 non-null   float64
 1   peserta.SUM(fkrtl.jml_kunjungan_fkrtl)          17994 non-null  float64
 2   peserta.SUM(fkrtl.lama_hari_kunjungan)          17994 non-null  float64
 3   peserta.status_peserta                          17994 non-null  float64
 4   peserta.SUM(fktp.jarak_hari_antar_kunjungan)    17994 non-null  float64
 5   peserta.provinsi_faskes                         17994 non-null  float64
 6   peserta.MEAN(fktp.jarak_hari_antar_kunjungan)   15287 non-null  float64
 7   peserta.segmen_peserta                          17994 non-null  float64
 8   peserta.STD(fkrtl.spesifikasi_kelompok_kasus)   5416 non-null   float64
 9   peserta.MAX(fkrtl.tarif_drugs)       

In [134]:
feature_store_peserta.isnull().sum()

peserta.STD(fkrtl.lama_hari_kunjungan)            12578
peserta.SUM(fkrtl.jml_kunjungan_fkrtl)                0
peserta.SUM(fkrtl.lama_hari_kunjungan)                0
peserta.status_peserta                                0
peserta.SUM(fktp.jarak_hari_antar_kunjungan)          0
peserta.provinsi_faskes                               0
peserta.MEAN(fktp.jarak_hari_antar_kunjungan)      2707
peserta.segmen_peserta                                0
peserta.STD(fkrtl.spesifikasi_kelompok_kasus)     12578
peserta.MAX(fkrtl.tarif_drugs)                        0
peserta.kab_kota_tempat_tinggal                       0
peserta.provinsi_tempat_tinggal                       0
peserta.SUM(fktp.no_keluarga)                         0
peserta.MEAN(fkrtl.spesifikasi_kelompok_kasus)        0
peserta.SUM(fkrtl.biaya_tagih)                        0
peserta.MAX(fktp.jarak_hari_antar_kunjungan)       2707
peserta.SUM(fkrtl.no_keluarga)                        0
peserta.MIN(fkrtl.spesifikasi_kelompok_kasus)   

In [135]:
feature_store_peserta = feature_store_peserta.fillna(0)

In [ ]:
feature_store_peserta.to_csv("../data/feature_store/feature_store_peserta.csv")

# Feature Selection for Training

In [136]:
selected_cols = ['peserta.STD(fkrtl.lama_hari_kunjungan)',
 'peserta.SUM(fkrtl.jml_kunjungan_fkrtl)',
 'jml_kunjungan_fkrtl',
 'status_pulang_peserta',
 'MONTH(tanggal_pulang)',
 'YEAR(tanggal_pulang)',
 'peserta.SUM(fkrtl.lama_hari_kunjungan)',
 'peserta.status_peserta',
 'peserta.SUM(fktp.jarak_hari_antar_kunjungan)',
 'MONTH(tanggal_datang)',
 'kode_nama_diagnosis_primer_ICD10',
 'kepemilikan_perujuk',
 'peserta.provinsi_faskes',
 'kode_casemix',
 'jenis_perujuk',
 'tipe_perujuk',
 'peserta.MEAN(fktp.jarak_hari_antar_kunjungan)',
 'peserta.segmen_peserta',
 'kepemilikan_fkrtl',
 'peserta.STD(fkrtl.spesifikasi_kelompok_kasus)',
 'kode_nama_diagnosis_ICD10',
 'peserta.MAX(fkrtl.tarif_drugs)',
 'peserta.kab_kota_tempat_tinggal',
 'kode_INACBGs',
 'tipe_fkrtl',
 'peserta.provinsi_tempat_tinggal',
 'peserta.SUM(fktp.no_keluarga)',
 'peserta.MEAN(fkrtl.spesifikasi_kelompok_kasus)',
 'peserta.SUM(fkrtl.biaya_tagih)',
 'kab_kota_fkrtl',
 'peserta.MAX(fktp.jarak_hari_antar_kunjungan)',
 'peserta.SUM(fkrtl.no_keluarga)',
 'peserta.MIN(fkrtl.spesifikasi_kelompok_kasus)',
 'peserta.SUM(fkrtl.bobot)',
 'peserta.STD(fkrtl.biaya_tagih)',
 'peserta.SUM(fktp.bobot)',
 'peserta.MIN(fkrtl.biaya_tagih)',
 'peserta.gender',
 'peserta.COUNT(fktp)']

In [137]:
final_data = feature_matrix_reduced[selected_cols]
final_data['readmitted_30d'] = feature_matrix_reduced['readmitted_30d']
final_data.head()

,peserta.STD(fkrtl.lama_hari_kunjungan),peserta.SUM(fkrtl.jml_kunjungan_fkrtl),jml_kunjungan_fkrtl,status_pulang_peserta,MONTH(tanggal_pulang),YEAR(tanggal_pulang),peserta.SUM(fkrtl.lama_hari_kunjungan),peserta.status_peserta,peserta.SUM(fktp.jarak_hari_antar_kunjungan),MONTH(tanggal_datang),...,peserta.MAX(fktp.jarak_hari_antar_kunjungan),peserta.SUM(fkrtl.no_keluarga),peserta.MIN(fkrtl.spesifikasi_kelompok_kasus),peserta.SUM(fkrtl.bobot),peserta.STD(fkrtl.biaya_tagih),peserta.SUM(fktp.bobot),peserta.MIN(fkrtl.biaya_tagih),peserta.gender,peserta.COUNT(fktp),readmitted_30d
id_kunjungan_fkrtl,,,,,,,,,,,,,,,,,,,,,
106840121V000074,1.000000,3.0,0,Sehat,1,2021,12.0,AKTIF,347.0,1,...,112.0,19768623.0,10.0,3.927782,1.148927e+04,10.474086,3411400.0,PEREMPUAN,8,1
107070121V000066,6.082763,3.0,0,Sehat,1,2021,15.0,AKTIF,327.0,1,...,75.0,202996110.0,10.0,37.224279,2.465712e+06,136.489023,3603800.0,PEREMPUAN,11,0
125360121V000104,0.707107,1.0,0,Sehat,1,2021,7.0,AKTIF,338.0,1,...,49.0,7678016.0,17.0,25.993140,9.607460e+05,402.893670,4296500.0,LAKI-LAKI,31,0
151020121V000038,1.414214,1.0,0,Sehat,1,2021,6.0,AKTIF,307.0,1,...,71.0,53673992.0,17.0,17.750227,1.545806e+06,97.626248,1295200.0,PEREMPUAN,11,0
155190121V000021,1.258306,6.0,0,Sehat,1,2021,13.0,AKTIF,308.0,1,...,88.0,216458904.0,15.0,12.372602,3.375500e+05,18.558904,3372600.0,PEREMPUAN,6,0


In [138]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27827 entries, 106840121V000074 to 77741221V010344
Data columns (total 40 columns):
 #   Column                                          Non-Null Count  Dtype   
---  ------                                          --------------  -----   
 0   peserta.STD(fkrtl.lama_hari_kunjungan)          15249 non-null  float64 
 1   peserta.SUM(fkrtl.jml_kunjungan_fkrtl)          27827 non-null  float64 
 2   jml_kunjungan_fkrtl                             27827 non-null  int64   
 3   status_pulang_peserta                           27827 non-null  category
 4   MONTH(tanggal_pulang)                           27827 non-null  category
 5   YEAR(tanggal_pulang)                            27827 non-null  category
 6   peserta.SUM(fkrtl.lama_hari_kunjungan)          27827 non-null  float64 
 7   peserta.status_peserta                          27827 non-null  category
 8   peserta.SUM(fktp.jarak_hari_antar_kunjungan)    27827 non-null  float64 
 9   MONTH(ta

In [139]:
final_data.to_parquet("../data/interim/final_data.parquet", engine='fastparquet')

In [140]:
train, test = train_test_split(
    final_data, test_size=0.2, random_state=42
)

In [141]:
train['readmitted_30d'].value_counts()

readmitted_30d
0    18735
1     3526
Name: count, dtype: int64

In [142]:
test['readmitted_30d'].value_counts()

readmitted_30d
0    4716
1     850
Name: count, dtype: int64

In [143]:
num_cols = list(train.select_dtypes("number").columns)
if "readmitted_30d" in num_cols:
    num_cols.remove("readmitted_30d")

cat_cols = list(train.select_dtypes("category").columns)

In [144]:
scaler = StandardScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test[num_cols] = scaler.transform(test[num_cols])

In [145]:
with open('../artifacts/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [146]:
for col in cat_cols:
    train[col] = train[col].cat.codes
    test[col] = test[col].cat.codes

In [147]:
train = train.fillna(0)
test = test.fillna(0)

In [148]:
train.head()

,peserta.STD(fkrtl.lama_hari_kunjungan),peserta.SUM(fkrtl.jml_kunjungan_fkrtl),jml_kunjungan_fkrtl,status_pulang_peserta,MONTH(tanggal_pulang),YEAR(tanggal_pulang),peserta.SUM(fkrtl.lama_hari_kunjungan),peserta.status_peserta,peserta.SUM(fktp.jarak_hari_antar_kunjungan),MONTH(tanggal_datang),...,peserta.MAX(fktp.jarak_hari_antar_kunjungan),peserta.SUM(fkrtl.no_keluarga),peserta.MIN(fkrtl.spesifikasi_kelompok_kasus),peserta.SUM(fkrtl.bobot),peserta.STD(fkrtl.biaya_tagih),peserta.SUM(fktp.bobot),peserta.MIN(fkrtl.biaya_tagih),peserta.gender,peserta.COUNT(fktp),readmitted_30d
id_kunjungan_fkrtl,,,,,,,,,,,,,,,,,,,,,
223150521V000299,0.227699,-0.094196,0.864951,0,4,2020,0.268705,2,-1.721583,4,...,0.000000,-0.356290,-0.504986,0.542504,-0.589181,-0.762970,-1.170089,1,-0.876887,0
270880121V001564,-0.298560,-0.225341,-0.489105,3,0,2020,-0.192478,2,-1.721583,0,...,0.000000,-0.346450,-0.823839,-0.005455,-0.906346,-0.671221,-0.605438,1,-0.784533,1
397270821V000663,0.020066,-0.094196,0.187923,3,7,2020,0.360941,1,0.433170,7,...,-0.478321,-0.172754,-0.823839,0.495776,-0.355748,1.089862,-0.454042,0,0.970200,0
37491221V000570,-0.422445,1.086116,2.896035,3,11,2020,1.652253,1,0.860893,11,...,-0.001438,-0.091115,-0.504986,2.748588,-0.092010,0.541366,-1.153514,1,0.323719,1
480960421V000720,-0.313880,4.823769,3.573063,3,3,2020,1.560016,2,-1.140526,3,...,-0.250247,0.746741,-0.823839,4.267781,-0.224923,-0.610887,-0.894251,1,-0.692178,1


In [149]:
train.to_csv("../data/interim/train.csv", index=False)
test.to_csv("../data/interim/test.csv", index=False)